# Foot Traffic Data Analysis

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import plotly.express as px

Reading in the data

In [ ]:
stores = pd.read_csv("data/db/stores.csv", dtype={"store_id":"string"})
customers = pd.read_csv("data/db/customers.csv", dtype={"customer_id":"string"})

visits = pd.read_csv("data/db/visits.csv", dtype={"customer_id":"string", "store_id":"string"})\
    .merge(stores, on="store_id", how="left")\
    .merge(customers, on="customer_id", how="left")

visits["visit_date"] = pd.to_datetime(visits["visit_date"])

display(stores.head())
display(customers.head())
display(visits.head())

## Calculating a time series of total visits by day

In [ ]:
visits_by_day = visits.groupby("visit_date").size()\
    .reset_index().rename(columns={0:"visit_count"})

visits_by_day

### Resampling for 2-week averages

In [ ]:
visits_by_day = visits.groupby("visit_date").size().resample("2W").mean()\
    .reset_index().rename(columns={0:"visit_count"})
visits_by_day

### Plotting Time Series

In [ ]:
visits_by_day.set_index("visit_date").plot()

## Calculating Gold, Silver, and Bronze Member Tiers

In [ ]:
members = visits[visits["is_member"] == True]
member_spend = members.groupby("customer_id").sum()["order_total"].reset_index()
member_spend

In [ ]:
member_spend["member_tier"] = pd.qcut(member_spend["order_total"], [0, 0.5, 0.90, 1], labels=["bronze", "silver", "gold"])
display(member_spend.head())
display(member_spend["member_tier"].value_counts())

In [ ]:
visits = visits.merge(member_spend[["customer_id", "member_tier"]], on="customer_id", how="left")
visits["member_tier"] = visits["member_tier"].astype("string").fillna("Nonmember").astype("category")
visits.head()

### Visualizing Member Tiers

In [ ]:
#pie chart
member_pie = visits["member_tier"].value_counts().reset_index().rename(columns={"index":"member_tier", "member_tier":"count"})
member_pie

In [ ]:
px.pie(member_pie, names="member_tier", values="count", hole=0.5)

In [ ]:
members = visits[visits["is_member"] == True]
mtiers = members.groupby("customer_id").sum()["order_total"].reset_index()
mtiers["membership_tier"] = pd.qcut(mtiers["order_total"], [0,0.5, 0.85, 0.95], labels=["bronze", "silver", "gold"])
df = visits.merge(mtiers[["customer_id", "membership_tier"]], on="customer_id", how="left")
df["membership_tier"] = df["membership_tier"].astype("string").fillna("Nonmember").astype("category")
df.groupby(["visit_date", "membership_tier"]).mean()["order_total"].unstack()#.plot()

In [ ]:
x = df["payment_method"].value_counts().to_frame().reset_index()
x.columns = ["payment_method", "count"]
x

px.pie(x, names='payment_method', values='count', hole=0.4)